In [ ]:
import datetime
import configparser
import os

import numpy as np
import pandas as pd

In [ ]:
DEGREE = 1
CONFIG_NAME = 'comb'
MESH_NAME = 'composite_fine'
MODEL_NAME = '4SM'
K = 9
ROMBERG_K = 5
GROUNDED_PLATE_AT = -0.088

FILENAME_PATTERN = f'FEM/solutions/paper/{MODEL_NAME}/{MESH_NAME}_{DEGREE}/{CONFIG_NAME}_sampled/{K}/{{name}}.npz'
ES_PREFIX = f'ES/paper/{MODEL_NAME}/{MESH_NAME}/{DEGREE}/{CONFIG_NAME}/'

In [ ]:
ELECTRODES = []
for col, x in zip('ABCDE', np.linspace(-6e-3, 6e-3, 5)):
    for row, z in enumerate(np.linspace(0.046, 0.076, 13)):
        ELECTRODES.append({'NAME': f'{col}_{row:02d}',
                           'X': x,
                           'Y': 0.0,
                           'Z': z})
ELECTRODES = pd.DataFrame(ELECTRODES)

In [ ]:
import _fast_reciprocal_reconstructor as frr
import common

In [ ]:
class Electrode_kESI(object):
    def __init__(self, filename, decimals_tolerance=None, dx=0):
        self.filename = filename
        self.decimals_tolerance = decimals_tolerance
        self.dx = dx
        with np.load(filename) as fh:
            self._X = self.round(fh['X'])
            self._Y = self.round(fh['Y'])
            self._Z = self.round(fh['Z'])
            self.x, self.y, self.z = fh['LOCATION']
#             try:
            self.base_conductivity = fh['BASE_CONDUCTIVITY']
#             except KeyError:
#                 pass
    
    def round(self, A):
        if self.decimals_tolerance is None:
            return A
        return np.round(A, decimals=self.decimals_tolerance)

    def correction_potential(self, X, Y, Z):
        _X, IDX_X, _ = np.intersect1d(self._X, self.round(X[:, 0, 0]), return_indices=True)
        assert len(_X) == np.shape(X)[0]
        _Y, IDX_Y, _ = np.intersect1d(self._Y, self.round(Y[0, :, 0]), return_indices=True)
        assert len(_Y) == np.shape(Y)[1]
        _Z, IDX_Z, _ = np.intersect1d(self._Z, self.round(Z[0, 0, :]), return_indices=True)
        assert len(_Z) == np.shape(Z)[2]

        with np.load(self.filename) as fh:
            return fh['CORRECTION_POTENTIAL'][np.ix_(IDX_X, IDX_Y, IDX_Z)]

    def base_potential(self, X, Y, Z):
        return (0.25 / (np.pi * self.base_conductivity)
                / (self.dx * 0.15
                   + np.sqrt(np.square(X - self.x)
                             + np.square(Y - self.y)
                             + np.square(Z - self.z))))

        
class Electrode_kCSD(object):
    def __init__(self, filename):
        with np.load(filename) as fh:
            self.x, self.y, self.z = fh['LOCATION']

In [ ]:
electrode = Electrode_kESI(FILENAME_PATTERN.format(name='A_00'))
# assert all(X == electrode._X)
# assert all(Y == electrode._Y)
# assert all(Z == electrode._Z)
XX = electrode._X
YY = electrode._Y
ZZ = electrode._Z

In [ ]:
dx = (XX[-1] - XX[0]) / (len(XX) - 1)
SRC_R_MAX = (2**(ROMBERG_K - 1)) * dx
ROMBERG_N = 2**ROMBERG_K + 1
print(SRC_R_MAX)

WIDE_R_MAX = 90e-3 - SRC_R_MAX
WIDE_R_MIN = 31e-3

NARROW_R_MAX = 79e-3
NARROW_R_MIN = 67e-3

E_Z_MAX = 76.1e-3
E_Z_MIN = 39.9e-3
E_X_DIST_MAX = 6.0001e-3


X_DIST_MAX = 9e-3
Y_DIST_MAX = 1e-3
print(E_Z_MIN, E_Z_MAX)

In [ ]:
X = XX[abs(XX) < X_DIST_MAX + SRC_R_MAX + dx]
Y = YY[abs(YY) < Y_DIST_MAX + SRC_R_MAX + dx]
Z = ZZ[(ZZ < max(WIDE_R_MAX, NARROW_R_MAX) + SRC_R_MAX + dx)
      & (ZZ > min(WIDE_R_MIN, NARROW_R_MIN) - SRC_R_MAX - dx)]

In [ ]:
ELE_WIDE = ELECTRODES[(ELECTRODES.Z >= E_Z_MIN)
                       & (ELECTRODES.Z <= E_Z_MAX)
                       & (abs(ELECTRODES.X) <= E_X_DIST_MAX)].copy()
print(len(ELE_WIDE))

_ELE_WIDE_R2 = np.square(ELE_WIDE.X) + np.square(ELE_WIDE.Y) + np.square(ELE_WIDE.Z)
ELE_NARROW = ELE_WIDE[(_ELE_WIDE_R2 >= np.square(NARROW_R_MIN))
                          & (_ELE_WIDE_R2 <= np.square(NARROW_R_MAX))].copy()
print(len(ELE_NARROW))

electrodes = {'kesi_narrow': 
                  [Electrode_kESI(FILENAME_PATTERN.format(name=name),
                                  decimals_tolerance=16,
                                  dx=dx)
                   for name in ELE_NARROW.NAME],
              'kcsd_narrow': 
                  [Electrode_kCSD(FILENAME_PATTERN.format(name=name))
                   for name in ELE_NARROW.NAME],
              'kesi_wide': 
                  [Electrode_kESI(FILENAME_PATTERN.format(name=name),
                                  decimals_tolerance=16,
                                  dx=dx)
                   for name in ELE_WIDE.NAME],
              'kcsd_wide': 
                  [Electrode_kCSD(FILENAME_PATTERN.format(name=name))
                   for name in ELE_WIDE.NAME],
              }

In [ ]:
# arc = np.arctan(abs(ELE.X / ELE.Z).max())
arc = np.arctan(E_X_DIST_MAX / NARROW_R_MIN)
print(f'{arc / np.pi * 180} deg')

In [ ]:
def inflate(A):
    B = np.empty(2 * len(A) - 1)
    B[::2] = A
    B[1::2] = 0.5 * (A[:-1] + A[1:])
    return B

In [ ]:
# convolver = frr.ckESI_convolver([X, Y, Z], [X, Y, Z])
convolver = frr.ckESI_convolver([X,
                                 Y,
                                 Z],
                                [inflate(X),
                                 inflate(Y),
                                 inflate(Z)])

In [ ]:
conductivity = 0.33
# sd = convolver.ds('POT')[0] * 16 / 6
sd = SRC_R_MAX / 3
model_src = common.SphericalSplineSourceKCSD(0, 0, 0,
                                             [sd, 3 * sd],
                                             [[1],
                                              [0,
                                               2.25 / sd,
                                               -1.5 / sd ** 2,
                                               0.25 / sd ** 3]],
                                             conductivity)
print(3 * sd)

In [ ]:
SRC_R2 = (np.square(convolver.SRC_X)
          + np.square(convolver.SRC_Y)
          + np.square(convolver.SRC_Z))
SRC_IDX_WIDE = ((SRC_R2 >= WIDE_R_MIN ** 2)
                 & (SRC_R2 <= WIDE_R_MAX ** 2)
                 & (abs(convolver.SRC_Y) < Y_DIST_MAX)
                 )

SRC_IDX_NARROW = ((SRC_R2 >= NARROW_R_MIN ** 2)
                    & (SRC_R2 <= NARROW_R_MAX ** 2)
                    & (abs(convolver.SRC_Y) < Y_DIST_MAX)
                    & (((convolver.SRC_X * 0
                        + convolver.SRC_Y * 0
                        + convolver.SRC_Z
                        / np.sqrt(SRC_R2)) >= np.cos(arc)))
                   )
source_idx = {'wide': SRC_IDX_WIDE,
              'narrow': SRC_IDX_NARROW}

In [ ]:
CSD_R2 = (np.square(convolver.CSD_X)
          + np.square(convolver.CSD_Y)
          + np.square(convolver.CSD_Z))
CSD_IDX = CSD_R2 <= 0.090 ** 2

In [ ]:
POT_R2 = (np.square(convolver.POT_X)
          + np.square(convolver.POT_Y)
          + np.square(convolver.POT_Z))
POT_ALLOWED = POT_R2 <= 0.090 ** 2

In [ ]:
CSD_IDX.mean(), SRC_IDX_WIDE.mean(), SRC_IDX_NARROW.mean()

# Kernels

In [ ]:
kernels = {f'{setup}__SRCS_{src}': frr.ckESI_kernel_constructor(
                                   model_src,
                                   convolver,
                                   SRC_IDX,
                                   CSD_IDX,
                                   eles,
                                   weights=ROMBERG_N)
           for setup, eles in electrodes.items()
           for src, SRC_IDX in source_idx.items()}

In [ ]:
# %%time
# kernels['kesi_naive_handicapped'] = kernels['kesi_naive']
# kernels['kesi_naive'] = frr.ckESI_kernel_constructor(
#                                 model_src,
#                                 convolver,
#                                 SRC_IDX_NAIVE,
#                                 CSD_IDX,
#                                 electrodes['kesi_naive'],
#                                 weights=ROMBERG_N,
#                                 leadfield_allowed_mask=POT_ALLOWED,
#                                 csd_allowed_mask=CSD_IDX) #,

# Eigenanalysis

In [ ]:
_SRC = np.zeros(convolver.shape('SRC'))
_CSD = np.zeros(convolver.shape('CSD'))

for name, ks in kernels.items():
    _n = len(ks._pre_kernel)
    _PHI = ks._pre_kernel * _n
    _KERNEL = ks.kernel * _n

    _EIGENVALUES, _EIGENVECTORS = np.linalg.eigh(_KERNEL)
    _EIGENVALUES, _EIGENVECTORS = _EIGENVALUES[::-1], _EIGENVECTORS[:, ::-1]
    _EIGENSOURCES = np.matmul(_PHI,
                              np.matmul(_EIGENVECTORS,
                                        np.diag(1. / np.sqrt(_EIGENVALUES))))

    for i, _ES in enumerate(_EIGENSOURCES.T):
        print(name, i)
        _SRC[ks.source_indices] = _ES
        _CSD[CSD_IDX] = convolver.base_weights_to_csd(_SRC,
                                                      model_src.csd,
                                                      (ROMBERG_N * 2 - 1,)*3)[CSD_IDX]
        np.savez_compressed(os.path.join(ES_PREFIX,
                                         f'{name}_ES{i:02d}.npz'),
                            X=convolver.CSD_X.flatten(),
                            Y=convolver.CSD_Y.flatten(),
                            Z=convolver.CSD_Z.flatten(),
                            CSD=_CSD,
                            EIGENSOURCE=_ES,
                            EIGENVECTOR=_EIGENVECTORS[:, i],
                            EIGENVALUE=_EIGENVALUES[i],
                            SOURCE_X=convolver.SRC_X.flatten(),
                            SOURCE_Y=convolver.SRC_Y.flatten(),
                            SOURCE_Z=convolver.SRC_Z.flatten(),
                            SOURCE_IDX=ks.source_indices)
    else:
        _SRC[ks.source_indices] = 0